In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import os

### Loading Price Data

In [2]:
project_root = os.path.abspath(os.getcwd())

data_dir = os.path.join(project_root, 'data')

# Summer month

p_summer = pd.read_csv(os.path.join(data_dir, 'Spotmarket_August_2024.csv'),sep=';')

# Winter month

p_winter = pd.read_csv(os.path.join(data_dir, 'Spotmarket_December_2024.csv'),sep=';')

#Spotmarket data from: https://energy-charts.info/charts/price_spot_market/chart.htm?l=en&c=CH&interval=month&year=2024&legendItems=by4&month=12


### Loading Fixed Appliances Data

In [3]:
#TV consumption data for summer month
tv_summer = pd.read_csv(os.path.join(data_dir, 'tv_consumption_august_2024_detailed.csv'), sep=';')

#TV consumption data for winter month
tv_winter = pd.read_csv(os.path.join(data_dir, 'tv_consumption_december_2024_detailed.csv'), sep=';')

#Lighting consumption data for summer month
lighting_summer = pd.read_csv(os.path.join(data_dir, 'lighting_consumption_august_2024.csv'), sep=';')

#Lighting consumption data for winter month
lighting_winter = pd.read_csv(os.path.join(data_dir, 'lighting_consumption_december_2024.csv'), sep=';')

#Fridge consumption data for summer month
fridge_summer = pd.read_csv(os.path.join(data_dir, 'fridge_August_2024.csv'), sep=';')

#Fridge consumption data for winter month
fridge_winter = pd.read_csv(os.path.join(data_dir, 'fridge_December_2024.csv'), sep=';')

#Oven consumption data for summer month
oven_summer = pd.read_csv(os.path.join(data_dir, 'Oven_Energy_Consumption_August_2024.csv'),sep=';')

#Oven consumption data for winter month
oven_winter = pd.read_csv(os.path.join(data_dir, 'Oven_Energy_Consumption_December_2024.csv'),sep=';')

#Induction stove consumption data for summer month
induction_summer = pd.read_csv(os.path.join(data_dir, 'Induction_Stove_Energy_Consumption_August_2024.csv'),sep=';')

#Induction stove consumption data for winter month
induction_winter = pd.read_csv(os.path.join(data_dir, 'Induction_Stove_Energy_Consumption_December_2024.csv'),sep=';')


### Data formatting

In [4]:
#get column names
p_summer.columns = p_summer.columns.str.replace(' ', '_')
p_winter.columns = p_winter.columns.str.replace(' ', '_')
tv_summer.columns = tv_summer.columns.str.replace(' ', '_')
tv_winter.columns = tv_winter.columns.str.replace(' ', '_')
lighting_summer.columns = lighting_summer.columns.str.replace(' ', '_')
lighting_winter.columns = lighting_winter.columns.str.replace(' ', '_')
fridge_summer.columns = fridge_summer.columns.str.replace(' ', '_')
fridge_winter.columns = fridge_winter.columns.str.replace(' ', '_')
oven_summer.columns = oven_summer.columns.str.replace(' ', '_')
oven_winter.columns = oven_winter.columns.str.replace(' ', '_')
induction_summer.columns = induction_summer.columns.str.replace(' ', '_')
induction_winter.columns = induction_winter.columns.str.replace(' ', '_')

### Optimization

In [5]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Load data
price = p_summer["Price (EUR/MWh)"].values
fridge = fridge_summer["Energy_kWh_1h"].values
#stove = df["Stove"].values
tv = tv_summer["tv_power_kWh"].values
lighting = lighting_summer["lighting_power_kWh"].values

T = len(price)

# Dishwasher properties
duration = 3  # hours of operation
min_gap = 15  # hours between runs
power_dishwasher = 1.5  # kW during operation

# Gurobi model
m = gp.Model("automated_demand_response")

# Binary start variables: 1 if dishwasher starts at hour t
s = m.addVars(T - duration + 1, vtype=GRB.BINARY, name="start")

# Binary on variables: 1 if dishwasher is on at hour t
x = m.addVars(T, vtype=GRB.BINARY, name="on")

# Link start and operation: if s[t]=1, then x[t] to x[t+2] = 1
for t in range(T - duration + 1):
    for k in range(duration):
        m.addConstr(x[t + k] >= s[t], name=f"link_{t}_{k}")

# Enforce max 1 run per day
hours_per_day = 24
days = T // hours_per_day
for d in range(days):
    m.addConstr(gp.quicksum(s[t] for t in range(d * 24, min((d + 1) * 24 - duration + 1, T - duration + 1))) <= 1,
                name=f"max_one_run_day_{d}")

# Enforce minimum gap (15 hours) between two starts
for t in range(T - duration + 1 - min_gap):
    m.addConstr(gp.quicksum(s[t + offset] for offset in range(1, min_gap + 1)) <= (1 - s[t]) * min_gap,
                name=f"min_gap_after_{t}")

# Total power consumption including fixed and dishwasher
total_load = [fridge[t] + stove[t] + tv[t] + lighting[t] + power_dishwasher * x[t] for t in range(T)]

# Objective: minimize total electricity cost
m.setObjective(gp.quicksum(price[t] * total_load[t] for t in range(T)), GRB.MINIMIZE)

# Optimize
m.optimize()

# Output dishwasher schedule
if m.status == GRB.OPTIMAL:
    print(f"Total cost: {m.ObjVal:.2f}")
    for t in range(T):
        if s.get(t) and s[t].X > 0.5:
            print(f"Dishwasher starts at: {df['Timestamp'][t]}")


KeyError: 'Price (EUR/MWh)'